# Llama Modelini bitsandbytes ile 8-Bit/4-Bit Kuantizasyon + Kaydetme

Bu not defteri, [*bitsandbytes*](https://github.com/TimDettmers/bitsandbytes) kullanarak *Hugging Face Transformers* üzerinden bir Llama modelini 8-bit veya 4-bit formatında **yükleyip** sonrasında **kaydetmeyi** gösterir.

## İçindekiler
1. [Kurulumlar](#kurulum)
2. [Modelin 8-Bit Olarak Yüklenmesi ve Kaydedilmesi](#8bit)
3. [Modelin 4-Bit Olarak Yüklenmesi ve Kaydedilmesi](#4bit)
4. [Perplexity/Accuracy Testi (Opsiyonel)](#perplexity)
5. [Örnek Üretim (Inference)](#inference)

> **Uyarı**: Bu not defterinin GPU (CUDA) destekli bir ortamda çalıştığından emin olunuz. `bitsandbytes`, çoğunlukla NVIDIA GPU ortamlarında sorunsuzdur.

## 1. Kurulumlar <a id='kurulum'></a>

Bu hücreyi çalıştırarak gerekli paketleri yükleyeceğiz. Ardından runtime'ı yeniden başlatmanız önerilir.


In [ ]:
%%bash
pip install --upgrade pip
pip install torch
pip install transformers accelerate
pip install bitsandbytes  # 4-bit ve 8-bit için

# İsteğe bağlı olarak evaluate paketiyle perplexity ölçümü yapmak isterseniz:
pip install datasets evaluate


## 2. Modelin 8-Bit Olarak Yüklenmesi ve Kaydedilmesi <a id='8bit'></a>

Aşağıdaki kod, `bitsandbytes`ı kullanarak bir Llama modelini 8-bit formatında yükler **ve** sonrasında `save_pretrained` ile diske kaydeder.

### Adımlar
1. **Tokenizer** ve **Model**’i `from_pretrained` ile indir veya yerel dizinden yükle.
2. `load_in_8bit=True` parametresini ekle.
3. `device_map='auto'` diyerek, GPU kullanılabilirliğine göre otomatik atama yap.
4. Kaydetme adımı: `model_8bit.save_pretrained("my-llama-8bit")` + `tokenizer.save_pretrained("my-llama-8bit")`.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Örneğin, Llama 2 7B model (veya Llama 3.3 / 3.8B benzeri bir model) için:
model_name = "meta-llama/Llama-2-7b-hf"  # Kendi model yolunuzu girin.

# 1. Tokenizer yükle
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Modeli 8-bit olarak yükle
model_8bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,       # 8-bit quantization
    device_map="auto"       # GPU otomatik seçilir
)

print("Model successfully loaded in 8-bit!")

# 3. Kaydetme
# Bu adım, 8-bit modelin parametrelerini local dizine kaydeder.
# Not: Tekrar yükleme esnasında load_in_8bit parametresini koyman gerekebilir.
model_8bit.save_pretrained("my-llama-8bit")
tokenizer.save_pretrained("my-llama-8bit")
print("\nModel & tokenizer saved to 'my-llama-8bit' folder!")

## 3. Modelin 4-Bit Olarak Yüklenmesi ve Kaydedilmesi <a id='4bit'></a>

Eğer 8-bit’ten daha az VRAM kullanımı istiyorsanız, 4-bit kuantizasyon deneyebilirsiniz. **Ancak**, 4-bit şu anda daha deneysel ve bazı modellerde hata verebilir.

### Adımlar
1. Tekrar `AutoModelForCausalLM.from_pretrained` çağır.
2. Bu kez `load_in_4bit=True` parametresini kullan.
3. Kaydetme adımında yine `model_4bit.save_pretrained("my-llama-4bit")`.

> **Not**: 4-bit’te bazı GPU’larda katman yükleme hataları yaşanabiliyor. Sürümlerin (CUDA, PyTorch, bitsandbytes) uyumlu olması önemli.

In [ ]:
# 1. Tokenizer yeniden yüklenebilir veya aynısını kullanabilirsiniz.
tokenizer_4bit = AutoTokenizer.from_pretrained(model_name)

# 2. Modeli 4-bit olarak yükle
model_4bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,       # 4-bit quantization
    device_map="auto"
)

print("Model successfully loaded in 4-bit!")

# 3. Kaydetme
model_4bit.save_pretrained("my-llama-4bit")
tokenizer_4bit.save_pretrained("my-llama-4bit")

print("\nModel & tokenizer saved to 'my-llama-4bit' folder!")

## 4. Perplexity veya Accuracy Testi (Opsiyonel) <a id='perplexity'></a>

Makaledeki yaklaşıma benzer şekilde, modele basit bir **perplexity** ölçümü uygulayabilirsiniz. Bu, modelin "metni ne kadar iyi tahmin ettiği"nin bir göstergesidir.

### Adımlar
1. [Hugging Face `datasets`](https://github.com/huggingface/datasets) kütüphanesinden bir test verisi çekin (ör. `wikitext`).
2. [Hugging Face `evaluate`](https://github.com/huggingface/evaluate) ile perplexity metriğini hesaplayın.

Aşağıdaki örnek, WikiText-2 ile kabaca bir test yapar (metinde *padding* / *truncation* vb. ayarlar basit tutulmuştur). **Gerçek senaryolarda** *data collator* ve ileri seviye ayarlara ihtiyaç duyabilirsiniz.

In [ ]:
import evaluate
from datasets import load_dataset

# Perplexity metriği
perplexity_metric = evaluate.load("perplexity")

# WikiText-2 test veri setini yükleyelim (küçük bir örnek)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test[:1%]")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

result_8bit = perplexity_metric.compute(
    model=model_8bit,
    tokenizer=tokenizer,
    inputs=tokenized_dataset
)

print("Perplexity (8-bit model):", result_8bit["perplexity"])

## 5. Örnek Üretim (Inference) <a id='inference'></a>

Son olarak, modelden bir prompt’a karşılık nasıl çıktı alabileceğinizi gösteren basit bir örnek.

### Adımlar
1. `generate` fonksiyonunu çağırın.
2. `temperature`, `max_new_tokens` gibi parametreleri ihtiyacınıza göre ayarlayın.
3. Çıktıyı `tokenizer.decode(...)` ile string’e çevirin.

In [ ]:
prompt = "Bana kısaca kuantizasyonun ne olduğunu anlat."  # Örnek Türkçe prompt
inputs = tokenizer(prompt, return_tensors="pt").to(model_8bit.device)

with torch.no_grad():
    generated_tokens = model_8bit.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True
    )

output = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
print("\n--- Model Output (8-bit) ---\n", output)

Aynı şekilde 4-bit model için örnek:

In [ ]:
prompt_4bit = "Kuantizasyon yöntemleri ve avantajları nelerdir?"
inputs_4bit = tokenizer_4bit(prompt_4bit, return_tensors="pt").to(model_4bit.device)

with torch.no_grad():
    generated_tokens_4bit = model_4bit.generate(
        **inputs_4bit,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True
    )

output_4bit = tokenizer_4bit.decode(generated_tokens_4bit[0], skip_special_tokens=True)
print("\n--- Model Output (4-bit) ---\n", output_4bit)

## Sonuç ve Öneriler

- Bu not defterinde, **bitsandbytes** kütüphanesi ile **8-bit** ve **4-bit** kuantizasyonun nasıl yapılacağını ve sonrasında bu modellerin nasıl kaydedileceğini gösterdik.
- 8-bit kuantizasyon, genelde **daha stabil** sonuç verir. 4-bit ile VRAM daha da düşer ama **hatalar** veya **tahmin kalitesinde** biraz kayıp gözlemlenebilir.
- Kaydettiğin klasörden tekrar yüklerken `from_pretrained("my-llama-8bit", load_in_8bit=True)` gibi bir kodla modele erişebilirsin (aynı parametreleri ekleyerek).
- Daha gelişmiş veya kalıcı (offline) kuantizasyon için [**GPTQ**](https://github.com/PanQiWei/Auto-GPTQ) veya [**llama.cpp**](https://github.com/ggerganov/llama.cpp) yöntemleri de kullanılabilir.

### Kaynaklar
- [Towards Data Science: Quantize Llama 3.8B with bitsandbytes](https://towardsdatascience.com/quantize-llama-3-8b-with-bitsandbytes-to-preserve-its-accuracy-e84283b233f7)
- [bitsandbytes GitHub Repo](https://github.com/TimDettmers/bitsandbytes)
- [Transformers Belgeleri](https://huggingface.co/docs/transformers)
- [Evaluate Kütüphanesi (perplexity)](https://github.com/huggingface/evaluate)